In [ ]:
import os
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
openai.api_key = OPENAI_API_KEY

In [ ]:
SANDBOX_DIR = "/Users/aidand/dev/babyagi/sandbox"
def create_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Created file {os.path.join(SANDBOX_DIR, path)}")

# Does the same thing as create_file - but nice to have a separate function for updating files
# So the LLM has the option to update files if it wants to - if that makes more sense than creating a new file
def update_file(path, content):
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Updated file {os.path.join(SANDBOX_DIR, path)}")

def delete_file(path):
    os.remove(os.path.join(SANDBOX_DIR, path))
    print(f"Deleted file {os.path.join(SANDBOX_DIR, path)}")


TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to update",
                },
                "content": {
                    "type": "string",
                    "description": "The content of the file to update",
                },
            },
            "required": ["path", "content"],
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_file",
            "description": "Update a file with the given name and content",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to delete",
                },
            },
            "required": ["path"],
        },
    },
]

In [ ]:
LOOP_LIMIT = 10

task_list = []

for i in range(LOOP_LIMIT):
    # Coding agent creates a step by step plan
    # Coding agent prioritizes the plan
    # Coding agent executes the plan

    # Review agent reviews the code
    # Review agent prioritizes the comments
